In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import Machine, generate_objects, BunchTracker, track, compare_traces, BeamTracker, plot3Dtraces
from test_tools import compare_beam_projections, plot_debug_data

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import Register, TurnDelay, TurnFIRFilter
from PyHEADTAIL.feedback.processors.convolution import Lowpass, Gaussian, FIRFilter
from PyHEADTAIL.feedback.processors.resampling import DAC, HarmonicADC, BackToOriginalBins, Upsampler
from PyHEADTAIL.feedback.processors.addition import NoiseGenerator
np.random.seed(0)

# 009 Injection error damping

This tests injection error damping by using a model for the original LHC damper system (ADT). The model still in progress and might not be correct!

## Basic parameters and elements for the simulations

In [ ]:
%%capture

n_macroparticles = 1000
n_slices = 10
n_segments = 5
n_sigma_z = 3
# n_sigma_z = 6

# n_turns = 100
n_turns = 70

machine = Machine(n_segments= n_segments)

# harmonic frequency of the bunches (f_RF/every_n_bucket_filled)
f_RF = 1./(machine.circumference/c/(float(machine.h_RF)))
f_harmonic = f_RF/10.


first_index = 10 #[buckets]
batch_spacing = 8  #[buckets]
# batch_spacing = 10  #[buckets]
n_batches = 4
# n_batches = 3
n_bunches_per_batch = 72
# n_bunches_per_batch = 6
bunch_spacing = 1 #[buckets]

batch_separation = batch_spacing+n_bunches_per_batch* bunch_spacing

filling_scheme = []
for j in xrange(n_batches):
    for i in xrange(n_bunches_per_batch):
        filling_scheme.append(first_index + i * bunch_spacing + j*batch_separation)


print filling_scheme

beam_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, n_slices,n_sigma_z,
                                                             filling_scheme=filling_scheme, matched=False)

In [ ]:
print 'f_RF: ' + str(f_RF)
print 'f_harmonic: ' + str(f_harmonic)
print('Number of bunches: ' + str(len(beam_ref.split())))

## Initial bunch kick
Creates an artificially  kicked beam which will be damped by using a feedback model.

The kick values are from numerical data:

In [ ]:
# numerical data for the injection error
kick_data = [
0.012621969679326828, -0.026454903239327976,
0.31558417363359004, 0.05072075638667073,
0.5941291539554268, -0.003539740574277417,
0.8730933140821122, 0.04746046901562728,
0.9238140704687821, 0.7840059616683348,
0.9512703476863598, 7.678582240748934,
0.9848047320742408, 8.099438764816842,
1.0260939428518192, 8.467618360075452,
1.0758715446775802, 8.967327262988753,
1.109510724016673, 9.414498963694372,
1.1523718590624337, 10.177406208518665,
1.211581006497287, 9.045481008826064,
1.2395612584709244, 8.071633171095225,
1.3254931184648697, 7.650078014019238,
1.4445401830418474, 7.544118674460307,
1.5336158915721563, 7.912018816515684,
1.5767914114715538, 8.753871591253116,
1.612526489834889, 9.727346824712978,
1.6383060478330722, 10.200880277590183,
1.7261242169488362, 10.252998299993013,
1.7713956358724756, 9.621154607484687,
1.8247362660394488, 9.015579516080203,
1.890023520644605, 9.40993456137491,
1.9316271162758198, 9.857059686546657,
2.019130900537947, 9.830232179036354,
2.0498358212431, 9.540578933886032,
2.1372348105540153, 9.487436249738012,
2.210696071353717, 9.934375072774273,
2.291597773689481, 10.2496914370881,
2.412007172632215, 10.48582939381943,
2.5316830069164666, 10.53776111408677,
2.610593605179199, 10.353089122284064,
2.6757760648331423, 10.721128990941061,
2.748818145827996, 11.062807107426469,
3.15521296662863, 11.11306210847442,
4.031089168859598, 11.055308446473068,
4.48516569246175, 11.078968817680074,
5.0192007638387555, 11.181108963461494,
5.2267995621899805, 11.311473882769382,
5.4422579818821175, 11.415477049905686,
5.816690342563051, 11.43960317645141,
6.150776647027316, 11.332386297477935,
6.525104212757038, 11.33019724738595,
6.866945343610999, 11.170303439603178,
7.186045970051932, 11.300016301436855,
7.465848489788312, 11.561537924128457,
7.761160662304091, 11.717705689201463,
8.064646841014413, 11.926457232016023,
8.310076616753218, 11.556600917538018,
8.531508348664445, 11.160569153023918,
8.702428914091428, 10.080622249132531,
8.791923802426584, 8.553783097738759,
8.825982161570527, 7.106215504995227,
8.930881907733866, 5.447707319345149,
8.966407396194779, 4.368552199529589,
8.982860203535084, 2.5000349316504042,
9.009163736289327, 1.1051442677161702,
8.981602664120537, 0.18425281199785815,
]

kick_data = np.stack((kick_data[0::2], kick_data[1::2]), axis=-1)
kick_data[:,0] = (kick_data[:,0]-0.95)*1e-6
kick_data[:,1] = kick_data[:,1]/10.

In [ ]:
bunch_list = beam_ref.split()

n_bunches = len(bunch_list)

amplitude = 0.003
f_osc = 1e6
wavelength = 1./f_osc*c

kick_x = 0.003*(-1.0+2*np.random.rand(n_bunches))
kick_y = 0.003*(-1.0+2*np.random.rand(n_bunches))

for i in xrange(n_bunches):
    if i > (n_bunches_per_batch-1):
        bunch_list[i].x = bunch_list[i].x + amplitude
        bunch_list[i].y = bunch_list[i].y + amplitude
    
#     bunch_list[i].x = bunch_list[i].x + amplitude * np.sin(2. * np.pi * bunch_list[i].mean_z() / wavelength)
#     bunch_list[i].y = bunch_list[i].y + amplitude * np.sin(2. * np.pi * bunch_list[i].mean_z() / wavelength)

t0 = bunch_list[n_bunches_per_batch].mean_z()/c
for i in xrange(n_bunches):
    if i > (n_bunches_per_batch-1):
        bunch_list[i].x = bunch_list[i].x + amplitude*np.interp(bunch_list[i].mean_z()/c-t0, kick_data[:,0],kick_data[:,1])
        bunch_list[i].y = bunch_list[i].y + amplitude*np.interp(bunch_list[i].mean_z()/c-t0, kick_data[:,0],kick_data[:,1])


beam_ref = sum(bunch_list)

## Special signal processor: SignalHolder

Special signal processor, which holds the signal between the gaps of the injected batches

In [ ]:
from PyHEADTAIL.feedback.core import default_macros

class SignalHolder(object):
    def __init__(self, hold_pattern, **kwargs):
        self._hold_pattern = hold_pattern
        
        self.extensions = []
        self._macros = [] + default_macros(self, 'SignalHolder', **kwargs)
        
        self._value_maps = []
        self._hold_maps = []
        
        value_indexes = self._hold_pattern[self._hold_pattern > 0]
        
        for i, idx in enumerate(value_indexes):
            self._value_maps.append(np.where(self._hold_pattern==idx)[0][0])
            self._hold_maps.append(np.where(self._hold_pattern==-idx)[0])
        
        
    def process(self, parameters, signal, *args, **kwargs):
        signal_out = np.copy(signal)
        
        for source_idx, target_map in zip(self._value_maps, self._hold_maps):
            signal_out[target_map] = signal[source_idx]
            
        return parameters, signal_out
        

Determines holding scheme for the SignalHolder

In [ ]:
extra_adc_bins = 10

max_bucket = np.max(filling_scheme)
min_bucket = np.min(filling_scheme)

n_bins = (max_bucket - min_bucket) + 1 
print n_bins
n_bins = n_bins + 2*extra_adc_bins
print n_bins
holding_scheme = np.zeros(int(n_bins))

gap_1_value_idx = int(extra_adc_bins + 2*(n_bunches_per_batch)+ batch_spacing/10) - 1
gap_1_from = gap_1_value_idx + 1
gap_1_to = gap_1_from + int(batch_spacing/10)

gap_2_value_idx = int(extra_adc_bins + 3*(n_bunches_per_batch)+ 2*batch_spacing/10) - 1
gap_2_from = gap_2_value_idx + 1
gap_2_to = gap_2_from + int(batch_spacing/10)


holding_scheme[gap_1_value_idx] = 1
holding_scheme[gap_1_from:gap_1_to] = -1

holding_scheme[gap_2_value_idx] = 2
holding_scheme[gap_2_from:gap_2_to] = -2

## Feedback settings

In [ ]:
feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# delay (a number of turns) before the pickup signal is used to the correction kick calculations.
delay = 1

# a number of values used to calculate the correct signal
n_values = 2

RMS_noise_level = 1e-6


# feedback settings
fc=1e6 # The cut off frequency of the power amplifier
ADC_f = 40e9 # multiplier of the sampling rate from the harmonic frequency
ADC_n_samples = 50
ADC_bits = 16
ADC_range = (-3e-3,3e-3)

DAC_bits = 14
DAC_range = (-3e-3,3e-3)

RMS_noise_level = 1e-6

## Reference data
Tracks a bunch by using an ideal bunch-by-bunch feedback presented in the first test (001_ideal_feedbacks.ipynb). The data is used as baseline data for the detailed feedback model. 

In [ ]:
beam_ref_data = copy.deepcopy(beam_ref)
tracker_ref_data = BeamTracker(beam_ref_data)
slicer_ref_data = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_ref_data,processors_bunch_x,processors_bunch_y, mpi=True,
                              circumference=machine.circumference, h_bunch=machine.h_bunch)
one_turn_map = [i for i in trans_map] + [feedback_map] #  + [long_map]

track(n_turns, beam_ref_data,one_turn_map ,tracker_ref_data)

## Detailed feedback model
Signal processors required for a detailed model of the transverse feedback system are tested here. Note that the tested system does not describe accurately any existing system, and the details of the models might affect significantly to the simulation results.

### FIR filter coefficients for the bandwidth phase correction
The bandwidth of this model is limited by using a 1st order lowpass filter, which has non-linear phase response. The phase of the filter is linearized by using a FIR filter. The filter coefficients are from Ref.  https://accelconf.web.cern.ch/accelconf/e08/papers/thpc122.pdf . Note that this is a test example and the coefficients are designed to the intrabunch feedback nor tuned to work perfectly with this model.

In [ ]:
# Coefficients from 
FIR_filter = [0.0096,  0.0192,  0.0481,  0.0673,  0.0769,  0.1154,
                0.1442,  0.1442,  0.2115,  0.2403,  0.2596,  0.3077,
                0.3558,  0.3846,  0.4519,  0.5192,  0.6346,  0.75,
                0.9519,  1.2019,  1.6346,  2.6346,  7.0192, -5.1923,
                -1.4135, -0.6827, -0.3942, -0.2308, -0.1442, -0.096,
                -0.0192, -0.0096]
FIR_filter = np.array(FIR_filter)
FIR_filter = FIR_filter/sum(FIR_filter)

### Turn-by-turn FIR filter for the betatron phase correction
The betatron phase correction is implemented manually by using a turn-by-turn FIR filter. Coefficients are from Ref. http://accelconf.web.cern.ch/AccelConf/IPAC2011/papers/mopo013.pdf, but they are not optimized to this model.

In [ ]:
# the total (group) delay to the middle coefficient is four turns, i.e.

turn_notch_filter = [-1.,1.]

phase_shift_x = -5. * machine.Q_x * 2.* pi
turn_phase_filter_x = [-2. * np.sin(phase_shift_x)/(pi * 3.),
                   0,
                   -2. * np.sin(phase_shift_x)/(pi * 1.),
                   np.cos(phase_shift_x),
                   2. * np.sin(phase_shift_x)/(pi * 1.),
                   0,
                   2. * np.sin(phase_shift_x)/(pi * 3.)
                   ]

phase_shift_y = -5. * machine.Q_y * 2.* pi
turn_phase_filter_y = [-2. * np.sin(phase_shift_y)/(pi * 3.),
                   0,
                   -2. * np.sin(phase_shift_y)/(pi * 1.),
                   np.cos(phase_shift_y),
                   2. * np.sin(phase_shift_y)/(pi * 1.),
                   0,
                   2. * np.sin(phase_shift_y)/(pi * 3.)
                   ]


turn_FIR_filter_x = np.convolve(turn_notch_filter, turn_phase_filter_x)
turn_FIR_filter_y = np.convolve(turn_notch_filter, turn_phase_filter_y)

# turn_FIR_filter_x = turn_phase_filter_x
# turn_FIR_filter_y = turn_phase_filter_y

### The model
This model includes elements for digital signal processing (ADC, FIR filters and DAC) and power amplifier/kicker bandwidth limitation. A model for pickup could be build, but in this test a charge weighted signal (𝛥-signal) is used (read more about this choise from the previous test).

In [ ]:
beam_detailed = copy.deepcopy(beam_ref)
tracker_detailed = BeamTracker(beam_detailed)
slicer_detailed = copy.deepcopy(slicer_ref)

processors_detailed_x = [
        Bypass(debug=False),
        ChargeWeighter(normalization = 'segment_average', debug=False),
#         NoiseGenerator(RMS_noise_level, debug=False),
        HarmonicADC(1*f_RF/10., ADC_bits, ADC_range,
                    n_extras=extra_adc_bins, debug=False),
        TurnFIRFilter(turn_FIR_filter_x, machine.Q_x, delay=1, debug=True),
        SignalHolder(holding_scheme, debug=True),
        Upsampler(2, [1,1], debug=True),
        FIRFilter(FIR_filter, zero_tap = 23, debug=False),
        DAC(ADC_bits, ADC_range, debug=False),
        Lowpass(fc, f_cutoff_2nd=4*fc, debug=False),
        BackToOriginalBins(debug=False),
]

processors_detailed_y = [
        Bypass(debug=True),
        ChargeWeighter(normalization = 'segment_average', debug=False),
#         NoiseGenerator(RMS_noise_level, debug=False),
        HarmonicADC(1*f_RF/10., ADC_bits, ADC_range,
                    n_extras=extra_adc_bins, debug=False),
        TurnFIRFilter(turn_FIR_filter_y, machine.Q_y, delay = 1, debug=True),
        SignalHolder(holding_scheme, debug=True),
        Upsampler(2, [1,1], debug=True),
        FIRFilter(FIR_filter, zero_tap = 23, debug=False),
        DAC(ADC_bits, ADC_range, debug=False),
        Lowpass(fc, f_cutoff_2nd=4*fc, debug=False),
        BackToOriginalBins(debug=False),
]


feedback_map = OneboxFeedback(feedback_gain,slicer_detailed,
                              processors_detailed_x,processors_detailed_y, mpi = True,
                              circumference=machine.circumference, h_bunch=machine.h_bunch)
one_turn_map = [feedback_map] + [i for i in trans_map] # + [long_map]

track(n_turns, beam_detailed, one_turn_map, tracker_detailed)
plot_debug_data(processors_detailed_x, source = 'output')

In [ ]:
compare_traces([tracker_ref_data,tracker_detailed],
               ['Ideal bunch-by-slice\nfeedback', 'Damper model'], bunch_idx=-10)
compare_beam_projections([ beam_ref_data, beam_detailed], 
               ['Ideal bunch-by-slice\nfeedback', 'Damper model'])

Jani Komppula, CERN, 2017

In [ ]:


plot3Dtraces(tracker_ref_data,
               ['Ideal bunch-by-slice\nfeedback'], bunches = 130,first_turn=0)

plot3Dtraces(tracker_detailed,
               ['Minimal model'], bunches = [100,120,203])